# Tutorial 06, case 4b: Poisson control problem with Neumann control and boundary observation

In this tutorial we solve the optimal control problem

$$\min J(y, u) = \frac{1}{2} \int_{\Gamma_2} (y - y_d)^2 ds + \frac{\alpha}{2} \int_{\Gamma_2} u^2 ds$$
s.t.
$$\begin{cases}
               - \Delta y = f     & \text{in } \Omega\\
    \epsilon \partial_n y = 0     & \text{on } \Gamma_1\\
    \epsilon \partial_n y = u     & \text{on } \Gamma_2\\
    \epsilon \partial_n y = 0     & \text{on } \Gamma_3\\
                        y = 0     & \text{on } \Gamma_4
\end{cases}$$

where
$$\begin{align*}
& \Omega               & \text{unit square}\\
& \Gamma_1             & \text{bottom boundary of the square}\\
& \Gamma_2             & \text{left boundary of the square}\\
& \Gamma_3             & \text{top boundary of the square}\\
& \Gamma_4             & \text{right boundary of the square}\\
& u \in L^2(\Gamma_2)  & \text{control variable}\\
& y \in H^1(\Omega)    & \text{state variable}\\
& \alpha > 0           & \text{penalization parameter}\\
& y_d                  & \text{desired state}\\
& f                    & \text{forcing term}
\end{align*}$$
using an adjoint formulation solved by a one shot approach.

Note that this is an alternative, to Lagrange multipliers, weak imposition of $y = y_d$ on $\Gamma_2$.

In [ ]:
import numpy as np
from mpi4py import MPI
from petsc4py import PETSc
import ufl
from ufl import grad, inner, Measure, replace, SpatialCoordinate, TestFunction, TrialFunction
from dolfinx import Constant, DirichletBC, Function, FunctionSpace
from dolfinx.fem import (apply_lifting, assemble_matrix, assemble_matrix_block, assemble_scalar,
                         assemble_vector, assemble_vector_block, BlockVecSubVectorWrapper,
                         create_vector_block, DofMapRestriction, locate_dofs_topological, set_bc)
from dolfinx.io import XDMFFile
from dolfinx.plotting import plot

### Mesh

In [ ]:
with XDMFFile(MPI.COMM_WORLD, "data/square.xdmf", "r") as infile:
    mesh = infile.read_mesh()
    mesh.topology.create_connectivity_all()
    subdomains = infile.read_meshtags(mesh, name="subdomains")
    boundaries = infile.read_meshtags(mesh, name="boundaries")
boundaries_2 = boundaries.indices[boundaries.values == 2]
boundaries_4 = boundaries.indices[boundaries.values == 4]

In [ ]:
# Define associated measures
dx = Measure("dx")(subdomain_data=subdomains)
ds = Measure("ds")(subdomain_data=boundaries)

### Function spaces

In [ ]:
Y = FunctionSpace(mesh, ("Lagrange", 2))
U = FunctionSpace(mesh, ("Lagrange", 2))
Q = Y.clone()

### Restrictions

In [ ]:
dofs_Y = np.arange(0, Y.dofmap.index_map.size_local + Y.dofmap.index_map.num_ghosts)
dofs_U = locate_dofs_topological(U, boundaries.dim, boundaries_2)
dofs_Q = dofs_Y
restriction_Y = DofMapRestriction(Y.dofmap, dofs_Y)
restriction_U = DofMapRestriction(U.dofmap, dofs_U)
restriction_Q = DofMapRestriction(Q.dofmap, dofs_Q)
restriction = [restriction_Y, restriction_U, restriction_Q]

### Trial and test functions

In [ ]:
(y, u, p) = (TrialFunction(Y), TrialFunction(U), TrialFunction(Q))
(z, v, q) = (TestFunction(Y), TestFunction(U), TestFunction(Q))

 ### Problem data

In [ ]:
alpha = 1.e-5
y_d = 1.
x = SpatialCoordinate(mesh)
ff = 10 * ufl.sin(2 * ufl.pi * x[0]) * ufl.sin(2 * ufl.pi * x[1])
bc0 = Function(Y)

### Optimality conditions

In [ ]:
a = [[y * z * ds(2), None, inner(grad(p), grad(z)) * dx],
     [None, alpha * u * v * ds(2), - p * v * ds(2)],
     [inner(grad(y), grad(q)) * dx, - u * q * ds(2), None]]
f = [y_d * z * ds(2),
     None,
     ff * q * dx]
a[2][2] = Constant(mesh, 0.) * p * q * dx
f[1] = Constant(mesh, 0.) * v * dx
bdofs_Y_4 = locate_dofs_topological((Y, Y), mesh.topology.dim - 1, boundaries_4)
bdofs_Q_4 = locate_dofs_topological((Q, Y), mesh.topology.dim - 1, boundaries_4)
bc = [DirichletBC(bc0, bdofs_Y_4, Y),
      DirichletBC(bc0, bdofs_Q_4, Q)]

### Solution

In [ ]:
(y, u, p) = (Function(Y), Function(U), Function(Q))

### Cost functional

In [ ]:
J = 0.5 * inner(y - y_d, y - y_d) * ds(2) + 0.5 * alpha * inner(u, u) * ds(2)

### Uncontrolled functional value

In [ ]:
# Extract state forms from the optimality conditions
a_state = replace(a[2][0], {q: z})
f_state = replace(f[2], {q: z})
bc_state = [bc[0]]

In [ ]:
# Assemble the linear system for the state
A_state = assemble_matrix(a_state, bcs=bc_state)
A_state.assemble()
F_state = assemble_vector(f_state)
apply_lifting(F_state, [a_state], [bc_state])
F_state.ghostUpdate(addv=PETSc.InsertMode.ADD, mode=PETSc.ScatterMode.REVERSE)
set_bc(F_state, bc_state)

In [ ]:
# Solve
ksp = PETSc.KSP()
ksp.create(mesh.mpi_comm())
ksp.setOperators(A_state)
ksp.setType("preonly")
ksp.getPC().setType("lu")
ksp.getPC().setFactorSolverType("mumps")
ksp.setFromOptions()
ksp.solve(F_state, y.vector)
y.vector.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)

In [ ]:
J_uncontrolled = mesh.mpi_comm().allreduce(assemble_scalar(J), op=MPI.SUM)
print("Uncontrolled J =", J_uncontrolled)
assert np.isclose(J_uncontrolled, 0.5295424)

In [ ]:
plot(y, title="uncontrolled state")

### Optimal control

In [ ]:
# Assemble the block linear system for the optimality conditions
A = assemble_matrix_block(a, bcs=bc, restriction=(restriction, restriction))
A.assemble()
F = assemble_vector_block(f, a, bcs=bc, restriction=restriction)

In [ ]:
# Solve
yup = create_vector_block(f, restriction=restriction)
ksp = PETSc.KSP()
ksp.create(mesh.mpi_comm())
ksp.setOperators(A)
ksp.setType("preonly")
ksp.getPC().setType("lu")
ksp.getPC().setFactorSolverType("mumps")
ksp.setFromOptions()
ksp.solve(F, yup)
yup.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)

In [ ]:
# Split the block solution in components
with BlockVecSubVectorWrapper(yup, [Y.dofmap, U.dofmap, Q.dofmap], restriction) as yup_wrapper:
    for yup_wrapper_local, component in zip(yup_wrapper, (y, u, p)):
        with component.vector.localForm() as component_local:
            component_local[:] = yup_wrapper_local

In [ ]:
J_controlled = mesh.mpi_comm().allreduce(assemble_scalar(J), op=MPI.SUM)
print("Optimal J =", J_controlled)
assert np.isclose(J_controlled, 8.0770609e-6)

In [ ]:
plot(y, title="state")

In [ ]:
plot(u, title="control")

In [ ]:
plot(p, title="adjoint")